In [1]:
import os

import QIGen
import torch
import time
import numpy as np

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

path = "../models/path" # PATH TO YOUR MODELS 

tokenizer = AutoTokenizer.from_pretrained(path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(path)

/local/home/tommaso/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:36<00:00,  1.12it/s]


In [3]:
checkpoint = torch.load("../checkpoint/path.pt",map_location=torch.device('cpu'))

In [4]:
l1 = 2**18 # Level 1 cache in bits
p = 64 # number of cores
bits = 3 # bits used for the model
gs = 128 # Group size (-1 means full column)
arch = 'llama' #or opt

qzeros = not (bits == 3)
qmodel = QIGen.swap_modules(arch, model, checkpoint, bits=bits, p=p, gs=gs,
                                     l1=l1, inplace=True, hint=1,
                                     verbose=False, qzeros=qzeros)

In [5]:
prompt = "My favorite vacation was "
result_length = 128
inputs = tokenizer(prompt, return_tensors="pt")
generation_config = transformers.GenerationConfig(
    temperature=0.8, #0.8
    top_p=0.95, #0.95
    top_k=40, #40
    num_beams=1,
    min_new_tokens=result_length,
    max_new_tokens=result_length,
    do_sample=False,
    repetition_penalty=1.1, #1.1
)

In [6]:
torch.manual_seed(0)
start = time.time()
output = tokenizer.decode(qmodel.generate(inputs["input_ids"],generation_config=generation_config)[0])
end = time.time() - start
print(end, output)

29.422704935073853  ⁇  My favorite vacation was 10 years ago. I went to the beach with my family and friends. We stayed in a hotel for two days. It was very hot, but we had fun. We swam in the ocean and played volleyball on the beach. We also went shopping at the mall. We bought some souvenirs and gifts for our families. We were so tired after the trip, but it was worth it.
My favorite vacation was 10 years ago. I went to the beach with my family and friends. We stayed in a hotel for two days. It was very hot, but we had fun
